In [17]:
import json

json_file = '/shared/hzhangal/Projects/zero-event-extraction/quizlet6/quizlet6_data/extracted_events.json'
file_name = 'quizlet6_data/L0C04D4DS.rsd.txt'

with open(json_file) as f:
    extracted_events = json.load(f)
    print(extracted_events.keys())
    print("\n")
    print(extracted_events[file_name]['events'][1])

dict_keys(['quizlet6_data/L0C04CJ13.rsd.txt', 'quizlet6_data/L0C04D4DS.rsd.txt', 'quizlet6_data/L0C04D4DB.rsd.txt', 'quizlet6_data/L0C04D4RB.rsd.txt', 'quizlet6_data/L0C04AT6Y.rsd.txt', 'quizlet6_data/L0C04AT6W.rsd.txt', 'quizlet6_data/L0C04AT6B.rsd.txt', 'quizlet6_data/L0C04AT6J.rsd.txt', 'quizlet6_data/L0C04D4DA.rsd.txt'])


[{'trigger': {'position': [6, 7], 'type': 'Life_Infect_Unspecified'}, 'arguments': [], 'sentence': 'Multistate Outbreak of Salmonella Typhimurium Infections Linked to Peanut Butter, 2008-2009 (FINAL UPDATE)', 'tokens': ['Multistate', 'Outbreak', 'of', 'Salmonella', 'Typhimurium', 'Infections', 'Linked', 'to', 'Peanut', 'Butter', ',', '2008', '-', '2009', '(', 'FINAL', 'UPDATE', ')']}]
